# 多任务编程——线程

补充：

\_\_init\_\_.py的作用除了提供\_\_all\_\_列表，还可以引入对应模块的一些常用函数，这样在主模块中获取相应的类或函数就可以使用包打点的方法，而不再需要包.模块.对象的方式。

## 一、线程

Python中另一种实现多任务的方式。

进程执行代码的一个分支，每个线程执行代码都是经过CPU的调度，线程是CPU调度的基本单位，每个进程至少有一个线程（主线程）。

进程只负责向操作系统索要资源。

同一进程下的线程全局变量是可以共享的。

## 二、多线程的使用

In [ ]:
from threading import *

thread1 = Thread(group=None, target=, args=, kwargs=, name=)

启动同样也是start方法。

In [ ]:
from threading import *
from time import sleep


def dance():
    for i in range(5):
        print('跳舞中')
        sleep(0.5)


def sing():
    for i in range(5):
        print('唱歌中')
        sleep(0.5)


if __name__ == '__main__':
    thread1 = Thread(target=dance)
    thread2 = Thread(target=sing)

    thread1.start()
    thread2.start()


同样存在静态方法，获取当前线程对象：

```py
def dance():
    for i in range(5):
        print(threading.current_thread(), '跳舞中')
        sleep(0.5)
```

进程中自动生成的是主线程，自己创建的则是子线程。

## 三、线程执行带有参数的任务

位置参数以元组，关键字参数以字典。

In [ ]:
from threading import *


class Dog(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def print_info(self):
        print(f'name={self.name}, age={self.age}')


if __name__ == '__main__':
    lele = Dog('lele', 9)

    sub_thread = Thread(target=Dog.print_info, args=(lele,))

    sub_thread.start()


**传入函数或方法对象一定不能有小括号**

## 四、线程注意点

### 1.线程执行是无序的

和进程的无序一样，具体哪个线程执行是由CPU调度决定的。

**操作系统调度进程，CPU调度线程**

### 2.主线程会等待所有子线程结束

如果希望主线程结束时子线程也随之结束,可以设置守护主线程（线程中没有terminate）：

```py
from threading import *
from time import sleep


def task():
    while True:
        print('任务执行中')
        sleep(0.3)


if __name__ == '__main__':
    sub_thread = Thread(target=task)
    sub_thread.daemon = True
    sub_thread.start()

    print('over')
```

也可以直接在创建时加入守护主线程属性

```py
sub_thread = Thread(target=task, daemon=True)
```

### 3.同一进程下的多进程共享全局变量

In [1]:
from threading import *
from time import sleep


g_list = []


def add_data():
    for i in range(5):
        g_list.append(i)
        sleep(2)


def read_data():
    print(g_list)


if __name__ == '__main__':
    thread1 = Thread(target=add_data)
    thread2 = Thread(target=read_data)

    thread1.start()
    thread1.join()  # join就是让主线程等待这个子线程执行完任务

    thread2.start()


[0, 1, 2, 3, 4]


### 4，线程之间共享全局变量出现错误

In [3]:
from threading import *
from time import sleep


num = 0


def add_func1():
    global num

    for i in range(10000000):
        num += 1

    print(f'task1: num={num}')


def add_func2():
    global num

    for i in range(10000000):
        num += 1

    print(f'task2: num={num}')


if __name__ == '__main__':
    thread1 = Thread(target=add_func1)
    thread2 = Thread(target=add_func2)

    thread1.start()
    thread2.start()



task1: num=12215913
task2: num=12739777


得到的运行结果小于20000000，原因是在另一个线程没有将修改后的数据写回之后，就被线程读走。

避免这个问题可以使用线程同步，保证数据的正确性：协同步调，按预定的先后次序进行运行。

两种方式：

1.线程等待——join

2.互斥锁

## 五、互斥锁

对共享数据进行锁定，保证同一时刻只有一个线程对其进行操作。

多个线程去争抢共享数据，抢到的线程先执行（上锁部分的代码），没有抢到的线程需要等待，等互斥锁释放后，其他线程再去争抢。

In [5]:
from threading import *


g_num = 0


lock = Lock()  # 调用函数创建互斥锁，类似于一个全局变量


def add_func1():
    lock.acquire()  # 上锁

    global g_num

    for i in range(10000000):
        g_num += 1

    print(f'task1: num={g_num}')

    lock.release()  # 用完之后及时释放


def add_func2():
    lock.acquire()

    global g_num

    for i in range(10000000):
        g_num += 1

    print(f'task2: num={g_num}')

    lock.release()


if __name__ == '__main__':
    thread1 = Thread(target=add_func1)
    thread2 = Thread(target=add_func2)

    thread1.start()

    thread2.start()



task1: num=10000000
task2: num=20000000


下面的代码原理上也可以保证运算结果的准确性，但明显感觉时间开销更大。

In [8]:
from threading import *


g_num = 0


lock = Lock()  # 调用函数创建互斥锁，类似于一个全局变量


def add_func1():
    global g_num

    for i in range(10000000):
        lock.acquire()
        g_num += 1
        lock.release()

    print(f'task1: num={g_num}')


def add_func2():
    global g_num

    for i in range(10000000):
        lock.acquire()
        g_num += 1
        lock.release()

    print(f'task2: num={g_num}')

    

if __name__ == '__main__':
    thread1 = Thread(target=add_func1)
    thread2 = Thread(target=add_func2)

    thread1.start()

    thread2.start()


task2: num=19953653
task1: num=20000000


注意：**线程执行完任务就会被销毁。**

无论是线程同步的哪种方式，本质就是将多任务转化为单任务，来保证数据安全。

如火车站售票，对于票数的准确性需要得到保证。

还有一个注意点，只有上锁才会被制约，如果上面一个函数上锁了，另一个函数没有上锁，没有上锁的函数并不会去争抢、等待互斥锁，而是自顾自的执行。

因此，**要上锁，都上锁**。

## 六、死锁

一直等待对方释放锁的情形。

In [ ]:
from threading import *


g_num = 0


lock = Lock()  # 调用函数创建互斥锁，类似于一个全局变量


def add_func1():
    lock.acquire()  # 上锁

    global g_num

    for i in range(10000000):
        g_num += 1

    print(f'task1: num={g_num}')

    # lock.release()  # 用完之后及时释放


def add_func2():
    lock.acquire()

    global g_num

    for i in range(10000000):
        g_num += 1

    print(f'task2: num={g_num}')

    lock.release()


if __name__ == '__main__':
    thread1 = Thread(target=add_func1)
    thread2 = Thread(target=add_func2)

    thread1.start()

    thread2.start()


及时释放互斥锁，避免其他线程无意义的等待。

## 七、进程与线程的对比

### 1.关系对比

线程依附于进程，进程创建就默认创建一个线程（主线程）。

### 2.区别对别

进程之间不共线全局变量（拷贝，完全独立）

线程之间共享全局变量，但是要注意资源竞争问题，解决办法：等待或互斥锁。

创建进程的资源开销大于线程（工作都是线程做的，CPU执行的进程中的线程）。

进程是操作系统资源分配的基本单位，线程是CPU调度的基本单位。

多进程开发比单进程多线程开发稳定性强（进程之间相互独立，单进程一旦挂掉，多任务的多功能就全部失效）。

### 3.优缺点对比

进程可以多核运行，但资源开销大。

线程资源开销小，不能多核运行。


计算密集型相关的一般使用多进程，逻辑密集型使用多线程。